This notebook articulates the process of generating a table of metadata about figures in the translation of the manuscript.

Firstly, import the `manuscript.py` file from the `manuscript-object` repo.

In [1]:
import manuscript

We will also use some convenient constants from the `utils.py` file.

In [2]:
import utils

For parsing the XML of each entry, we will use the Python library BeautifulSoup.

[Read the documentation here.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [4]:
from bs4 import BeautifulSoup, Tag

Our task is essentially this:
- for each entry in the manuscript, obtain a list of figures in that entry
- for each of those figures, generate a row of spreadsheet data containing the following information:
    - entry ID
    - entry title
    - figure ID
    - figure size
    - figure link
    - figure margin data (if applicable)
    - entry categories
    - the terms present in the entry for each semantic tag
- write all of these rows to a CSV

We will begin with the smallest conceptual unit--the individual figure--and work up from there.

This first function `extract`s information from a figure element.

The actual data type being passed to this function is a `Tag` object parsed by BeautifulSoup. This kind of object is more intelligent than a string of XML; it possess information about its own attributes and the structure of the XML document. We take advantage of this when searching for margin metadata, since figures in the margin may encode their position as an attribute of the `figure` tag or as an attribute of their enclosing `ab` tag.

For the rest of the metadata, it suffices to simply request the attribute from the element.

The `size` attribute is not included if the figure has the default size `medium`. We would like to see this explicitly, though, so we tell it to default to `medium` if no `size` attribute was found.

Finally we return this metadata as a Python dictionary keyed by what will eventually become the names of the columns of our spreadsheet.

_Note:_ while it is theoretically possible in XML that there could be an intermediate enclosing tag between the `ab` and the `figure` which would ruin our ability to request the margin attribute from the enclosing `ab` tag, in fact the manuscript is encoded such that this does not happen.

So, the following _never_ happens:
```xml
<ab margin='blah'>
<intermediate_parent_tag>
<figure></figure>
</intermediate_parent_tag>
</ab>
```
This means we can trust that the margin metadata is either in the attributes of the figure tag, the attributes of its parent, or it is not there at all.

See the end of this notebook for proof of this fact.

In [19]:
def extract(figure):
    size = figure.get('size') or 'medium'
    fig_id = figure.get('id')
    link = figure.get('link')
    
    # Use a short-circuiting 'or' to search for margin metadata first in the tag's attributes,
    # then in the attributes of the enclosing `ab` tag, and then finally return None if neither one yielded data.
    margin = figure.get('margin') or figure.parent.get('margin')
        
    return {'size':size,
            'fig_id':fig_id,
            'margin':margin,
            'link':link}

The next conceptual stage to consider is the level of the entry. Each entry may contain several figures.

The actual data type being given as input to be `transform`ed into several rows of data is an `Entry` object as defined in `entry.py`. This custom class possess information about its ID, title, categories, properties (i.e. semantic tags), and XML contents, among other things. This metadata combined with what we get from each individual figure tag accounts for each of the columns we wish to represent in our spreadsheet.

After we get the basic metadata information, we parse the XML content of the entry using BeautifulSoup. This gives us a "souped" XML element representing the root. We use the `find_all` method to recursively get every `figure` element in the entry as a list. These are exactly the objects we pass to `extract` in order to get their particular metadata.

We use a list comprehension to `extract` the metadata from all of the figures, giving us all of the rows we need from this entry.
Then it remains to add to each of these rows the metadata about the entry itself. We do this in a `for` loop that simply adds more key-value pairs to the dictionary. We have a nested `for` loop which adds all of the semantic tags and associated lists of terms as individual columns.

In [6]:
def transform(entry):
    entry_id = entry.identity
    categories = entry.categories
    title = entry.title
    
    soup = BeautifulSoup(entry.xml_string, 'lxml')
    figures = soup.find_all('figure')
    data = [extract(fig) for fig in figures]
    
    for row in data:
        row['title'] = title
        row['entry_id'] = entry_id
        row['categories'] = categories
        for prop, tag in utils.prop_dict.items():
            row[tag] = entry.properties[prop]
            
    return data

With the ability to now transform an entry object into several rows of figure metadata, it remains to apply this transformation to each fo the entries in the manuscript.

First, we must generate the manuscript object and all of the entry objects from the `ms-xml/tl` path.

In [20]:
m = manuscript.Manuscript.from_dirs(utils.tl_path)

Generating Manuscript object for versions tl...
Generating entries from files in folder /mnt/c/code/github/m-k-manuscript-data/ms-xml/tl...
Found 4 divs in file with IDs: p001r_1, p001r_2, p001r_3, p001r_4.
Found 1 div in file with ID: p001v_1.
Found 2 divs in file with IDs: p002r_1, p002r_2.
Found 3 divs in file with IDs: p002v_1, p002v_2, p002v_3.
Found 3 divs in file with IDs: p003r_1, p003r_2, p003r_3.
Found 2 divs in file with IDs: p003r_3, p003v_1.
Found 2 divs in file with IDs: p004r_1, p004r_2.
Found 3 divs in file with IDs: p004v_1, p004v_2, p004v_3.
Found 3 divs in file with IDs: p004v_3, p005r_1, p005r_2.
Found 2 divs in file with IDs: p005r_2, p005v_1.
Found 1 div in file with ID: p006r_1.
Found 2 divs in file with IDs: p006v_1, p006v_2.
Found 4 divs in file with IDs: p007r_1, p007r_2, p007r_3, p007r_4.
Found 4 divs in file with IDs: p007v_1, p007v_2, p007v_3, p007v_4.
Found 4 divs in file with IDs: p008r_1, p008r_2, p008r_3, p008r_4.
Found 5 divs in file with IDs: p008v_1,

Found 4 divs in file with IDs: p085v_1, p085v_2, p085v_3, p085v_4.
Found 1 div in file with ID: p085v_4.
Found 2 divs in file with IDs: p085v_4, p086v_1.
Found 2 divs in file with IDs: p086v_1, p087r_1.
Found 3 divs in file with IDs: p087v_1, p087v_2, p087v_3.
Found 6 divs in file with IDs: p088r_1, p088r_2, p088r_3, p088r_4, p088r_5, p088r_6.
Found 3 divs in file with IDs: p088v_1, , p088v_2.
Found 3 divs in file with IDs: p089r_1, p089r_2, p089r_3.
Found 2 divs in file with IDs: p089v_1, p089v_2.
Found 3 divs in file with IDs: p090r_1, p090r_2, p090r_3.
Found 3 divs in file with IDs: p090r_3, p090v_1, p090v_2.
Found 2 divs in file with IDs: p091r_1, p091r_2.
Found 1 div in file with ID: p091r_2.
Found 4 divs in file with IDs: p092r_1, p092r_2, p092r_3, p092r_4.
Found 3 divs in file with IDs: p092v_1, p092v_2, p092v_3.
Found 5 divs in file with IDs: p093r_1, p093r_2, p093r_3, p093r_4, p093r_5.
Found 3 divs in file with IDs: p093v_1, p093v_2, p093v_3.
Found 1 div in file with ID: p094r

Generating entry with folio 005v, ID p005v_1...
Generating entry with folio 006r, ID p006r_1...
Generating entry with folio 006v, ID p006v_1...
Generating entry with folio 006v, ID p006v_2...
Generating entry with folio 007r, ID p007r_1...
Generating entry with folio 007r, ID p007r_2...
Generating entry with folio 007r, ID p007r_3...
Generating entry with folio 007r, ID p007r_4...
Generating entry with folio 007v, ID p007v_1...
Generating entry with folio 007v, ID p007v_2...
Generating entry with folio 007v, ID p007v_3...
Generating entry with folio 007v, ID p007v_4...
Generating entry with folio 008r, ID p008r_1...
Generating entry with folio 008r, ID p008r_2...
Generating entry with folio 008r, ID p008r_3...
Generating entry with folio 008r, ID p008r_4...
Generating entry with folio 008v, ID p008v_1...
Generating entry with folio 008v, ID p008v_2...
Generating entry with folio 008v, ID p008v_3...
Generating entry with folio 008v, ID p008v_4...
Generating entry with folio 008v, ID p00

Generating entry with folio 063v, ID p063v_6...
Generating entry with folio 063v, ID p063v_7...
Generating entry with folio 064r, ID p064r_1...
Generating entry with folio 064r, ID p064r_6...
Generating entry with folio 064r, ID p064r_2...
Generating entry with folio 064r, ID p064r_7...
Generating entry with folio 064r, ID p064r_3...
Generating entry with folio 064r, ID p064r_8...
Generating entry with folio 064r, ID p064r_4...
Generating entry with folio 064r, ID p064r_5...
Generating entry with folio 064v, ID p064v_1...
Generating entry with folio 064v, ID p064v_3...
Generating entry with folio 064v, ID p064v_4...
Generating entry with folio 064v, ID p064v_5...
Generating entry with folio 065r, ID p065r_1...
Generating entry with folio 065r, ID p065r_2...
Generating entry with folio 065r, ID p065r_4...
Generating entry with folio 065r, ID p065r_3...
Generating entry with folio 065v, ID p065v_1...
Generating entry with folio 065v, ID p065v_6...
Generating entry with folio 065v, ID p06

Generating entry with folio 093r, ID p093r_1...
Generating entry with folio 093r, ID p093r_2...
Generating entry with folio 093r, ID p093r_3...
Generating entry with folio 093r, ID p093r_4...
Generating entry with folio 093r, ID p093r_5...
Generating entry with folio 093v, ID p093v_1...
Generating entry with folio 093v, ID p093v_2...
Generating entry with folio 093v, ID p093v_3...
Generating entry with folio 094r, ID p094r_1...
Generating entry with folio 096v, ID p096v_1...
Generating entry with folio 096v, ID p096v_2...
Generating entry with folio 096v, ID p096v_3...
Generating entry with folio 097r, ID p097r_1...
Generating entry with folio 097r, ID p097r_2...
Generating entry with folio 097r, ID p097r_3...
Generating entry with folio 097v, ID p097v_1...
Generating entry with folio 097v, ID p097v_2...
Generating entry with folio 098r, ID p098r_1...
Generating entry with folio 098r, ID p098r_2...
Generating entry with folio 098r, ID p098r_3...
Generating entry with folio 098v, ID p09

Generating entry with folio 153r, ID p153r_1...
Generating entry with folio 153r, ID p153r_2...
Generating entry with folio 153v, ID p153v_1...
Generating entry with folio 153v, ID p153v_2...
Generating entry with folio 154r, ID p154r_1...
Generating entry with folio 154r, ID p154r_2...
Generating entry with folio 154r, ID p154r_3...
Generating entry with folio 154r, ID p154r_4...
Generating entry with folio 154r, ID p154r_5...
Generating entry with folio 154v, ID p154v_1...
Generating entry with folio 154v, ID p154v_2...
Generating entry with folio 154v, ID p154v_3...
Generating entry with folio 154v, ID p154v_4...
Generating entry with folio 154v, ID p154v_5...
Generating entry with folio 155r, ID p155r_1...
Generating entry with folio 155v, ID p155v_1...
Generating entry with folio 156r, ID p156r_1...
Generating entry with folio 156r, ID p156r_2...
Generating entry with folio 156r, ID p156r_3...
Generating entry with folio 156v, ID p156v_1...
Generating entry with folio 157r, ID p15

Generating folio from file /ms-xml/tl/tl_p047v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p048r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p048v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p049r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p049v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p050r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p050v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p051r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p051v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p052r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p052v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p053r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p053v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p054r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p054v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p055r_preTEI.xml...
Generating folio from fi

Generating folio from file /ms-xml/tl/tl_p124v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p125r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p125v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p126r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p126v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p127r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p127v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p128r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p128v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p129r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p129v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p130r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p130v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p131r_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p131v_preTEI.xml...
Generating folio from file /ms-xml/tl/tl_p132r_preTEI.xml...
Generating folio from fi

Then all we must do is iterate over the entries, transforming each one and extending our initially empty list of rows as we go.

In [22]:
rows = []
for entry in m.entries['tl'].values():
    rows.extend(transform(entry))

Since there are 162 figures in the manuscript, we should expect there to be 162 rows.

In [24]:
len(rows)

162

The most convenient way to write this data to a CSV file is using the DataFrame module from the pandas library.

In [25]:
from pandas import DataFrame

Here we define the order and names of our spreadsheet columns, which matches to the keys of our dictionaries.

Instead of painstakingly writing out each of the semantic tags, we cleverly use the `prop_dict` constant from `utils.py` and `*`-expand the abbreviated values into items in this list of column names.

In [26]:
columns=['entry_id', 'title', 'fig_id', 'link', 'size', 'margin', 'categories', *utils.prop_dict.values()]

The DataFrame module has a constructor method specifically designed to form a table from a list of dictionary records like we have. We provide it our data and our desired columns and it returns the table.

In [27]:
df = DataFrame.from_records(csv_data, columns=columns)

In [28]:
df

,entry_id,title,fig_id,link,size,margin,categories,al,bp,cn,...,sn,tl,tmp,wp,de,el,it,la,oc,po
0,3r3,Thick varnish for planks,fig_p003v_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,medium,None,[varnish],[],[],"[sous, sous, sous]",...,[],"[vessel, vessel, vessel, oven, copper vessel, ...","[months, times past, hour]",[],[],[],[],[pix græca],[],[warp]
1,6v1,For ga cach cages,fig_p006v_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,medium,None,[glass process],[],[hand],[],...,[],"[enamel cannules, cannules, cutting file, wood...",[],[],[],[],[],[],[],[]
2,14r1,For walls of earth and rustic construction,fig_p014r_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,small,left-middle,[household and daily life],[Swallows],[foot],[],...,[],"[ditch-spade, measuring line, long <-ch-> pole...",[],[],[],[],[],[],[],"[ditch-spade, tamp, tamps]"
3,16r1,Founding of soft iron,fig_p016r_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,large,left-middle,[metal process],[],[handfuls],[],...,[],"[small forges, iron pots, furnace, blast-pipe,...",[],[],[],[],[],[],[],[]
4,17r1,On the gunner,fig_p019r_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,x-small,None,[arms and armor],"[horses, horses, horses, horses, horses, good ...",[thumb],"[lb, lb, lb, lb]",...,"[bigger voice, sound]","[ladle, ladles, cauldrons, burin, burin, ladle...","[longer, day, by night]","[cannon, cannon-perrier, small, short cannons,...",[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,167r1,Petards,fig_p168r_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,small,left-middle,[arms and armor],"[oxen, mules]","[thumbs breadth, fingers, saliva, hand]",[],...,[],"[crucible, forks, pegs, tools, thick canvas, t...",[],"[Petards, Petards, petard, petard, petard, pet...",[],[],[],[],"[crucible, gimlet, gimlet, gimlets, gimlets]","[cake, presses, strap hinge]"
158,169v2,Reducing a round figure to a hollow form,fig_p169v_3,https://drive.google.com/open?id=0B9-oNrvWdlO5...,small,left-middle,[casting],[],[warm urine],[],...,[],"[clay slab, mold, mold, clay slab]",[],[],[],[],[],[],[],[]
159,170r6,Cleaning closed molds,fig_p170r_1,https://drive.google.com/open?id=0B9-oNrvWdlO5...,x-small,left-bottom,[casting],[],[],[],...,[],"[chaple, molds, thin wire of latten, delicate ...",[],[],[],[],[],[],[],[]
160,170r6,Cleaning closed molds,fig_p170r_2,https://drive.google.com/open?id=0B9-oNrvWdlO5...,x-small,None,[casting],[],[],[],...,[],"[chaple, molds, thin wire of latten, delicate ...",[],[],[],[],[],[],[],[]


Finally, this table can easily be written to a CSV.

In [29]:
df.to_csv('figures.csv', index=False)

The following query proves that there are no `ab` tags in the manuscript which have an intermediate tag between itself and a child `figure` tag.

In [30]:
a = m.generate_all_folios(method='xml', version='tl') # Generate one giant XML document to search representing the entire manuscript in translation.

Adding folio 1r to allFolios tl xml...
Adding folio 1v to allFolios tl xml...
Adding folio 2r to allFolios tl xml...
Adding folio 2v to allFolios tl xml...
Adding folio 3r to allFolios tl xml...
Adding folio 3v to allFolios tl xml...
Adding folio 4r to allFolios tl xml...
Adding folio 4v to allFolios tl xml...
Adding folio 5r to allFolios tl xml...
Adding folio 5v to allFolios tl xml...
Adding folio 6r to allFolios tl xml...
Adding folio 6v to allFolios tl xml...
Adding folio 7r to allFolios tl xml...
Adding folio 7v to allFolios tl xml...
Adding folio 8r to allFolios tl xml...
Adding folio 8v to allFolios tl xml...
Adding folio 9r to allFolios tl xml...
Adding folio 9v to allFolios tl xml...
Adding folio 10r to allFolios tl xml...
Adding folio 10v to allFolios tl xml...
Adding folio 11r to allFolios tl xml...
Adding folio 11v to allFolios tl xml...
Adding folio 12r to allFolios tl xml...
Adding folio 12v to allFolios tl xml...
Adding folio 13r to allFolios tl xml...
Adding folio 13v t

In [31]:
soup = BeautifulSoup(a, 'lxml') # Soupify the document.

In [32]:
all_ab = soup.find_all('ab') # Get a list of all the `ab` tags.

Technically, you should never write a list comprehension as horrible as the one you see below.
However, this is merely for querying purposes.
Essentially, it asks the following:

**"Give me every `ab` tag in the manuscript for which any of its descendants themselves have child tags which are `figure` tags."**

Or, put another way:

**"List those `ab` tags which have grandchildren (or great-grandchildren, or great-great-grandchildren...) that are `figure` tags."**

See? Not that bad at all.
It returns an empty list, meaning there are no such tags. Every figure tag inside an `ab` tag is a direct descendant with no intermediate parents.

In [36]:
[ab for ab in all_ab if any([len(list(c.children))>0 and any([d.name == 'figure' for d in c.children]) for c in ab.descendants if isinstance(c, Tag)])]

[]